In [1]:
import os
import astropy.io.fits as fits
import numpy as np
import scipy
import scipy.ndimage as ndi
import matplotlib.pylab as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pyklip.klip
import pyklip.instruments.Instrument as Instrument
import pyklip.parallelized as parallelized
import pyklip.rdi as rdi
import pyklip.fakes as fakes
import glob
from astropy.table import Table
from astropy.table import join
from astropy.table import vstack
import pdb

ModuleNotFoundError: No module named 'pyklip'

# Creating Contrast Curves

Using NIRCAM data simulated by the Space Telescope Science Institute, we'd like to estimate the contrast needed by a given planet at different separations from its host star in order for it to be detectable by the James Webb Space Telescope. This notebook performs an analysis of contrast in starlight-subtracted images in order to generate contrast curves for the visualization of this information.
The first step in this process is thus utilzing the pyKLIP algorithm to remove starlight, then measuring the contrast in the image at varying distances. However, this PSF subtraction can result in the distortion of planet spectrum, and an oversubtraction of flux from the image as a whole. Thereofore, we also measure the algortithm throughput as a function of planet sepration.

### Loading the dataset

This loads loads in our simulated JWST data, specifying the roll angle (10 arcseconds is the maximum roll angle) as well as the center as the image and the inner working angle. It combines images from both roll angles (0 and 10 arcseconds) to create a full sequence of images.

In [ ]:
#Import the dataset to be used
filtername = "f300m"

# read in roll 1
with fits.open("old_simulated_data/NIRCam_target_Roll1_{0}.fits".format(filtername)) as hdulist:
    roll1_cube = hdulist[0].data

# read in roll 2
with fits.open("old_simulated_data/NIRCam_target_Roll2_{0}.fits".format(filtername)) as hdulist:
    roll2_cube = hdulist[0].data  

# combine the two rows
full_seq = np.concatenate([roll1_cube, roll2_cube], axis=0)

# two rolls are offset 10 degrees
pas = np.append([0 for _ in range(roll1_cube.shape[0])], [10 for _ in range(roll2_cube.shape[0])])

# for each image, the (x,y) center where the star is is just the center of the image
centers = np.array([np.array(frame.shape)/2. for frame in full_seq])

# give it some names, just in case we want to refer to them
filenames = np.append(["roll1_{0}".format(i) for i in range(roll1_cube.shape[0])],
                      ["roll2_{0}".format(i) for i in range(roll1_cube.shape[0])])

#Define dataset
dataset = Instrument.GenericData(full_seq, centers, IWA=4, parangs=pas, filenames=filenames)
dataset.flipx = False

### Run Klip

Use the pyklip algortihm on the dataset in order to remove starlight from the images. Using KLIP-ADI reductions, we will break the image into 9 concentric annuli, and each annuli into 4 azimuthal sectors.

In [ ]:
parallelized.klip_dataset(dataset, outputdir="./", fileprefix="pyklip-f300m-ADI-k50a9s4m1", annuli=9, 
                          subsections=4, numbasis=[1,5,10,20,50], mode="ADI", movement=1)

### Mask Any Real Planets in Image

There are two "real" planets already in this dataset, but we don't want to include them in our noise estimation. Their positions are at (41,54) and (43,70) respectively. We will therefore mask these this positions with nans before calculating the contrast in our images. We used 5 KL modes to KLIP our data, but to create the contrast curve we only need 1 frame. We'll arbitrarily chose the KL mode with an index of 2 to accomplish this. 

In [ ]:
#Read in the KLIP-ed dataset
with fits.open("pyklip-f300m-ADI-k50a9s4m1-KLmodes-all.fits") as hdulist:
    adi_cube = hdulist[0].data
    adi_centers = [hdulist[0].header['PSFCENTX'], hdulist[0].header['PSFCENTY']]

    
#Plot the KL10 Cube (index of 2)
plt.figure() 
plt.imshow(adi_cube[2], interpolation='nearest', cmap='inferno')
plt.gca().invert_yaxis()

In [ ]:
#Mask the 1st planet
planet1_peak_x = 41 
planet1_peak_y = 54

#Create an array with the indices are that of KL mode frame with index 2
ydat, xdat = np.indices(adi_cube[2].shape)

#Set the FWHM of the PSF
psf_fwhm = 5

#Calculate the distance around the planet to be masked
distance_from_planet1 = np.sqrt((xdat - planet1_peak_x)**2 + (ydat - planet1_peak_y)**2)

#Mask
adi_cube[2][np.where(distance_from_planet1 <= 2*psf_fwhm)] = np.nan

#Mask the second planet
planet2_peak_x = 43
planet2_peak_y = 70
distance_from_planet2 = np.sqrt((xdat - planet2_peak_x)**2 + (ydat - planet2_peak_y)**2)
adi_cube[2][np.where(distance_from_planet2 <= 2*psf_fwhm)] = np.nan

#plot the new masked data
plt.figure()
plt.imshow(adi_cube[2], interpolation='nearest', cmap='inferno')
plt.gca().invert_yaxis()

### Measure the Contrast

Using the pyKLIP function meas_contrast, we can compute the 5 $\sigma$ noise at each separation in our image. For this function, we again need to specify our planet's FWHM as well as our outer working angle and the center of our input frame. 

In [ ]:
OWA = 65 #Setting this to be distance between star and outer edge of image

#Measuring the contrast in the image
contrast_seps, contrast = pyklip.klip.meas_contrast(dat = adi_cube[2], 
                                                     iwa = dataset.IWA, 
                                                     owa = OWA, 
                                                     resolution = (psf_fwhm)/2, 
                                                     center = adi_centers,
                                                     low_pass_filter = False)

In [ ]:
#Plot contrast curve!
plt.figure(figsize = (15,10))
plt.plot(contrast_seps, contrast, color = "teal")
plt.xlabel("Separation (pixels)")
plt.ylabel("Contrast (5 $\sigma$)")
plt.title("Non-Calibrated Contrast Curve")
plt.rcParams.update({'font.size': 22})
plt.savefig("contrast_adi2.png", dpic = 500)

## Computing throughput

Now that we've created our contrast curve, we can calculate the throughput of our KLIP reduced images. In order to optimize this calculation, we need to inject multiple fake planets at varying separations and postion angles to get a feel for how throughput changes across the image. Therefore, we'll create a loop that injects 10 planets at a set list of separations, but changing the position angle or "thetas" each time. Each time it injects a new set of fake planets, it runs pyKLIP on the injected dataset, and then retrieves the fluxes of the fake planets from the KLIP-ed data to be output into a table.  

### Injecting fake planets:

The injected fake planets will be scaled down versions of the unocculted PSF. The following code simply crops and centers this unocculted psf to prepare it for injection

In [ ]:
# read in unocculted PSF
with fits.open("old_simulated_data/NIRCam_unocculted_{0}.fits".format(filtername)) as hdulist:
    psf_cube = hdulist[0].data 
    psf_head = hdulist[0].header
    
# collapse reference psf in time
psf_frame = np.nanmean(psf_cube, axis=0)

# find the centroid
bestfit = fakes.gaussfit2d(psf_frame, 71, 30, searchrad=3, guessfwhm=2, guesspeak=1, refinefit=True)

psf_xcen, psf_ycen = bestfit[2:4]

# recenter PSF to that location
x, y = np.meshgrid(np.arange(-20,20.1,1), np.arange(-20,20.1,1))
x += psf_xcen
y += psf_ycen

psf_stamp = scipy.ndimage.map_coordinates(psf_frame, [y,x])

### Generating a new dataset

Since we're using a loop to inject fake planets in new locations each time, we also need to regenerate a new dataset each time. Here we'll generate 10 datasets.



In [ ]:
datasets = []
for dataset in range(10):
    
    #Import the dataset to be used
    filtername = "f300m"

    # read in roll 1
    with fits.open("old_simulated_data/NIRCam_target_Roll1_{0}.fits".format(filtername)) as hdulist:
        roll1_cube = hdulist[0].data

    # read in roll 2
    with fits.open("old_simulated_data/NIRCam_target_Roll2_{0}.fits".format(filtername)) as hdulist:
        roll2_cube = hdulist[0].data  

    # combine the two rows
    full_seq = np.concatenate([roll1_cube, roll2_cube], axis=0)

    # two rolls are offset 10 degrees
    pas = np.append([0 for _ in range(roll1_cube.shape[0])], [10 for _ in range(roll2_cube.shape[0])])

    # for each image, the (x,y) center where the star is is just the center of the image
    centers = np.array([np.array(frame.shape)//2. for frame in full_seq])

    # give it some names, just in case we want to refer to them
    filenames = np.append(["roll1_{0}".format(i) for i in range(roll1_cube.shape[0])],
                          ["roll2_{0}".format(i) for i in range(roll1_cube.shape[0])])

    #Define dataset
    dataset = Instrument.GenericData(full_seq, centers, IWA=4, parangs=pas, filenames=filenames)
    dataset.flipx = False
    datasets.append(dataset)

### Begin the loop:

1) Specify Values of thetas for planets to be injected at  
2) Specify input contrasts and planet separations  
3) Inject fake planets with  fakes.inject_planet  
4) Run KLIP on fake injected dataset  
5) Read in the KLIP-ed dataset  
6) Retrieved fluxes of injected planets with fakes.retrieve_planet_flux  
7) Append retrieved fluxes, planet separtions and contrasts to a list  

In [ ]:
def retrieve_multiple(thetas, planet_seps, planet_contrast, thetas_step_size):
    
    """This function injects fake planets into a dataset given a list of thetas 
    (position angles, separations and input contrast. It then run the starlight subtraction algorithm pyklip on 
    the data injected with fake planets, and retrieved the flux of the starlight subtracted images.
    
    Inputs:
    -------
    thetas: the angles (CCW) to inject the fake planets at 
    planet_sep: distance in pixels from the star to inject fake planets at
    input_contrast: the contrast to multiple the unocculted psf by before injecting
    thetas_step_size: amount by which to increase theta each loop. 
    
    Returns:
    --------
    
    A list of all retrieved fluxes, all planet separations, and all input contrast
    
    """
    
    retrievedfluxes_all = []
    planet_seps_all = []
    retrieved_contrast_all = []
    retrieved_contrast_seps_all = []
    retrieved_input_contrasts_all = []


In [ ]:
thetas = [30, 60, 90, 120, 150, 180, 210, 240]
retrievedfluxes_all = []
planet_seps_all = []
retrieved_contrast_all = []
retrieved_contrast_seps_all = []
retrieved_input_contrasts_all = []
thetas_step_size = 75
for dataset, i in zip(datasets,range(len(datasets))):

    #Specify the desired contrasts of the fake planets' flux
    input_contrasts = [3e-3, 2e-3, 1e-3, 3e-3, 2e-3, 1e-3, 3e-3, 2e-3]
    planet_seps = [10,12, 15,20,25,30,35,40]
    planet_seps_all.append(planet_seps)
    retrieved_input_contrasts_all.append(input_contrasts)
    planet_fwhm = 3.5

    #making more psf stamps
    psf_stamp_input = np.array([psf_stamp for i in range(12)])
    #print(psf_stamp.shape)

    #Defining angle of injected fakes in CCW angle from +x axis to +y axis
    thetas_values = [x+thetas_step_size*i for x in thetas]
    thetas_array = []
    for theta_value in thetas_values:
        these_thetas = np.ones(12)*theta_value
        corrected_thetas = these_thetas - dataset.PAs
        thetas_array.append(corrected_thetas)

    for input_contrast, planet_sep, theta in zip(input_contrasts, planet_seps, thetas_array):
    #Multiply unocculted psf by desired contrast level to simulate planet psf
        planet_fluxes = psf_stamp_input*input_contrast

            
        fakes.inject_planet(frames = dataset.input, 
                            centers=dataset.centers, 
                            inputflux=planet_fluxes, 
                            astr_hdrs=dataset.wcs, 
                            radius=planet_sep,
                            pa = None,
                            thetas = theta)

    #Image with fakes injected before KLIP
    plt.imshow(dataset.input[2], cmap = 'inferno')

    plt.figure()
    plt.imshow(dataset.input[8], cmap = 'inferno')

    plt.figure()
    plt.imshow(dataset.input[8] - dataset.input[2])
    
    #Set output directory
    outputdir = 'contrastcurves'
    fileprefix = 'FAKE_KLIP_ADI_A9K5S4M1' + str(i)
    numbasis = [1,5,10,20,50]


    #Run KLIP on dataset with injected fakes
    parallelized.klip_dataset(dataset, 
                              outputdir=outputdir, 
                              fileprefix=fileprefix, 
                              algo = 'klip', 
                              annuli=9, 
                              subsections=4, 
                              movement=1, 
                              numbasis=numbasis, 
                              mode="ADI")
    

    # replace os.path.join(foldername,filename)
    klipdataset = "contrastcurves/"+ fileprefix + "-KLmodes-all.fits"
    with fits.open(klipdataset) as hdulist:
        outputfile = hdulist[0].data
        outputfile_centers = [hdulist[0].header['PSFCENTX'], hdulist[0].header['PSFCENTY']]

    outputfile_frame = outputfile[2]
    
    thetas_retrieve = [x+thetas_step_size*i for x in thetas]
    retrieved_planet_fluxes = []

    
    #retrieving planet flux
    for input_contrast, planet_sep, theta in zip(input_contrasts, planet_seps, thetas_retrieve):
        print(planet_sep, theta)

        fake_flux = fakes.retrieve_planet_flux(frames = outputfile_frame, 
                                                centers=outputfile_centers,
                                                astr_hdrs=dataset.wcs[0], 
                                                sep=planet_sep,
                                                pa = None,
                                                thetas = theta,
                                                searchrad = 7)

        retrieved_planet_fluxes.append(fake_flux)

    retrievedfluxes_all.append(retrieved_planet_fluxes)

#Make tables of the flux and corresponding separation measuremetns



### Calculate throughput

We can first create a table using the lists of retrieved planets fluxes, their separations and input fluxes. Then, we'll add the column "throughput" which will hold the throughput calculations for each value in the table. The throughput calculation is: $\frac{Retrieved\ flux}{Input\ flux}$

In [ ]:
tables = []
#Loop through each list to create a table of all variables
for flux, sep, input_contrast_all in zip(retrievedfluxes_all, planet_seps_all, retrieved_input_contrasts_all):
    input_flux = np.array(input_contrast_all)*bestfit[0]
    t = Table([flux,sep, input_flux], names = ('flux', 'separation', 'input_flux'))
    
    tables.append(t)
flux_sep = vstack([x for x in tables])
#Calculate throughput and add it to list 
flux_sep['throughput'] = flux_sep['flux']/flux_sep['input_flux']

#We can also calculate the median througput per separation

#Group by separation
flux_by_sep = flux_sep.group_by('separation')

#Calculate the median value for each separation group
med_flux_by_sep = flux_by_sep.groups.aggregate(np.median)

In [ ]:
plt.figure(figsize = (15,10))
plt.scatter(flux_sep["separation"], flux_sep["throughput"], color = '#BC96E6', label = "Calculated Throughputs")
plt.plot(med_flux_by_sep["separation"], med_flux_by_sep["throughput"], color = 'red', label = "Median Throughput")
plt.ylabel("Throughput")
plt.xlabel("Planet Separation")
plt.title("Algorithm Throughput")
plt.legend(frameon = False)
plt.rcParams.update({'font.size': 22})
plt.savefig("throughput_med.png")

In [ ]:
#open all fake planets after pyklip to make sure
#they're not too close to the calibration planets
#add some code where it throws out values too close to real planets
#inject planets brighter
#try setting annuli to 1 and subsections to 1
#check that fake planets aren't overlapping
#clock by 75 instead